First we started by reading the csv file 'captions.txt' and then displaying the first five rows

In [41]:


import pandas as pd

captions_path = "../data/captions/captions.txt"

df = pd.read_csv(captions_path)
print(df.head())


                       image  \
0  1000268201_693b08cb0e.jpg   
1  1000268201_693b08cb0e.jpg   
2  1000268201_693b08cb0e.jpg   
3  1000268201_693b08cb0e.jpg   
4  1000268201_693b08cb0e.jpg   

                                             caption  
0  A child in a pink dress is climbing up a set o...  
1              A girl going into a wooden building .  
2   A little girl climbing into a wooden playhouse .  
3  A little girl climbing the stairs to her playh...  
4  A little girl in a pink dress going into a woo...  


Now we want group all rows that have the same image filename together, keeping only the caption column, then for each group we convert all captions into a python list, and finally convert the pandas object into a dictionary

In [42]:
image_to_captions = (
    df.groupby('image')['caption'].apply(list).to_dict()
)

print("Number of images: ", len(image_to_captions))

Number of images:  8091


Creating a new column for captions in lower case and without punctuation

In [43]:

import string

#Removing the upper cases
df['lower_caption'] = df['caption'].str.lower()

#Function to remove the punctuation
def remove_punctuation(lower_caption):
    for char in string.punctuation:
        lower_caption = lower_caption.replace(char, '')
    return lower_caption

df['lower_caption'] = df['lower_caption'].apply(remove_punctuation)

#Comparing the new column with the old column
df.sample(10)



,image,caption,lower_caption
120,1030985833_b0902ea560.jpg,A black dog and a brown dog are jumping up to ...,a black dog and a brown dog are jumping up to ...
39179,687513087_413d4a3a3b.jpg,Two large black dogs are snarling at each other .,two large black dogs are snarling at each other
37480,493621130_152bdd4e91.jpg,A brown dog leaps into the water .,a brown dog leaps into the water
30025,3511062827_cd87871c67.jpg,A black dog pulling on a purple object with it...,a black dog pulling on a purple object with it...
39522,763577068_4b96ed768b.jpg,A small dog with a red ribbon on its head walk...,a small dog with a red ribbon on its head walk...
16482,2894217628_f1a4153dca.jpg,The football player wearing a white helmet and...,the football player wearing a white helmet and...
35337,3741462565_cc35966b7a.jpg,A young girl dressed in pink with a hair barre...,a young girl dressed in pink with a hair barre...
3584,179829865_095b040377.jpg,a waterskier wearing a red life jacket is hold...,a waterskier wearing a red life jacket is hold...
16254,2884301336_dc8e974431.jpg,The white and black dog plays in a blue pool w...,the white and black dog plays in a blue pool w...
11127,2545192257_142fe9e2de.jpg,Two big dogs play in a room in a house .,two big dogs play in a room in a house


Tokenization for each caption

In [44]:

import nltk
from nltk.tokenize import word_tokenize
#Was having trouble with the import word_tokenize so had to add the line below
#nltk.download('punkt_tab')

#Creating a new column for tokens for each caption
df['tokens'] = df['lower_caption'].apply(lambda x: word_tokenize(x))
df.head()





,image,caption,lower_caption,tokens
0,1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set o...,a child in a pink dress is climbing up a set o...,"[a, child, in, a, pink, dress, is, climbing, u..."
1,1000268201_693b08cb0e.jpg,A girl going into a wooden building .,a girl going into a wooden building,"[a, girl, going, into, a, wooden, building]"
2,1000268201_693b08cb0e.jpg,A little girl climbing into a wooden playhouse .,a little girl climbing into a wooden playhouse,"[a, little, girl, climbing, into, a, wooden, p..."
3,1000268201_693b08cb0e.jpg,A little girl climbing the stairs to her playh...,a little girl climbing the stairs to her playh...,"[a, little, girl, climbing, the, stairs, to, h..."
4,1000268201_693b08cb0e.jpg,A little girl in a pink dress going into a woo...,a little girl in a pink dress going into a woo...,"[a, little, girl, in, a, pink, dress, going, i..."
